# Scikit-Learn binary classification model

## Remote train via Azure ML Compute (AML Cluster) and optional HyperDrive (Hyper-parameter tunning with multiple child runs)

- This notebook showcases the creation of a ScikitLearn Binary classification model by remotely training on Azure ML Compute Target (AMLCompute Cluster)

- It shows multiple ways of remote training like using a single Estimator, a ScriptRunConfig and hyper-parameter tunning with HyperDrive with multiple child trainings

In [ ]:
# This will work in Jupyter out of the box, but not in JupyterLab
# Hence we need to install the JupyterLab Widget Extension
# Afterwards, restart Jupyter by opening Jupyter and hitting the Quit button, then re-login to JupyterLab
!sudo -i jupyter labextension install @jupyter-widgets/jupyterlab-manager

## Check library versions

This is important when interacting with different executions between remote compute environments (cluster) and the Compute Instance/VM/Laptop/etc. with the Jupyter Notebook.
If not using the same versions you can have issues when creating .pkl files in the cluster and downloading them to load it in the Jupyter notebook.

In [ ]:
# Check versions
import azureml.core
import sklearn
import joblib
import pandas

print("Azure SDK version:", azureml.core.VERSION)
print('scikit-learn version is {}.'.format(sklearn.__version__))
print('joblib version is {}.'.format(joblib.__version__))
print('pandas version is {}.'.format(pandas.__version__))

## Setup and connect to AML Workspace

In [ ]:
from azureml.core import Workspace

#ws = Workspace.from_config()
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

ws = Workspace(subscription_id="3c8972d9-f541-46b2-b70b-d81baba3595d",
                  resource_group="amlworkshop",
                  workspace_name="amlworkshop",
                  auth=interactive_auth)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'classif-attrition-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the `max_nodes`, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

### Create project directory and copy the training script into the project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os
import shutil

project_folder = './classif-attrition-amlcompute'
os.makedirs(project_folder, exist_ok=True)

# Copy the training script into the project directory
shutil.copy('train.py', project_folder)

### Connect or Create a Remote AML compute cluster

Try to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`).
If not available, create a new cluster from the code.

In [ ]:
# Define remote compute target to use

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster - Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Alternative - Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 8)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)
    
# For a more detailed view of current AmlCompute status, use get_status().

### Fetch the AML Dataset

In [ ]:
aml_dataset = ws.datasets['IBM-Employee-Attrition']

## Create Environment

#### Optionally list all the available environments and packages in your AML Workspace

- Environments specify the Python packages, environment variables, and software settings around your training and scoring scripts.
- They also specify run times (Python, Spark, or Docker).
- The environments are managed and versioned entities within your Machine Learning workspace that enable reproducible, auditable, and portable machine learning workflows across a variety of compute targets.
- You can also persist environements file-based in `yaml` or `json`.

For more details, see [this link](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments).

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

# List Environments and packages in my workspace
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        #print("packages", envs[env].python.conda_dependencies.serialize_to_string())
        
# Use curated environment from AML named "AzureML-Tutorial"
curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")

# Custom environment: Environment.get(workspace=ws,name="myenv",version="1")

## Configure & Run using ScriptRunConfig & Environment 

The easiest option for running our code is using a `ScriptRunConfig` with an environement. This is especially easy to use for single node training jobs.

In [ ]:
# Add training script to run config
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment

script_runconfig = ScriptRunConfig(source_directory=project_folder, 
                            script="train.py",
                            arguments=[aml_dataset.as_named_input('attrition')]
                           )

# Attach compute target to run config
script_runconfig.run_config.target = aml_remote_compute
# runconfig.run_config.target = "local"

# Attach environment to run config
script_runconfig.run_config.environment = curated_environment

## (Option A) Run the experiment with a single ScriptRunConfig and single run

This will train the model with fixed parameters on a single node. If you want to use hyperparameter tuning, see option C.

In [ ]:
# Submit the Experiment Run to the AML Compute 
run = experiment.submit(script_runconfig)
run

The experiment will now show up in the AzureML studio UI.

In [ ]:
from azureml.widgets import RunDetails

# Show interactive widget for monitoring the run
RunDetails(run).show()

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

#### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use `wait_for_completion()` to show when the model training is finished:

In [ ]:
# This will wait until the run finished
run.wait_for_completion(show_output=True)  # specify True for a verbose log

## (Option B) Configure and Run with Intelligent hyperparameter tuning (HyperDrive using Estimator)

**IMPORTANT:** You need to have created the ScriptRunConfig in the previous steps (Without submitting the experiment). 

The adjustable parameters that govern the training process are referred to as the **hyperparameters** of the model. The goal of hyperparameter tuning is to search across various hyperparameter configurations and find the configuration that results in the best performance.

To demonstrate how Azure Machine Learning can help you automate the process of hyperarameter tuning, we will launch multiple runs with different values for numbers in the sequence. First let's define the parameter space using random sampling.

### Create a hyperparameter sweep
First, we will define the hyperparameter space to sweep over. 
In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, Accuracy. More configuration details can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters).

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BayesianParameterSampling 
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
    
# solver{'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, default='lbfgs'
# penalty{'l1', 'l2', 'elasticnet', 'none'}, default='l2' --- Note that some penalty parameters are not supported by some algorithms..
param_sampling = RandomParameterSampling( {
    "--solver": choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
    "--penalty": choice('l2')
    }
)

# Details on Scikit-Learn LogisticRegression hyper-parameters:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Now we will define an early termination policy. The BanditPolicy basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.


In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
# Note that early termination policy is currently NOT supported with Bayesian sampling
# Check here for recommendations on the multiple policies:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#picking-an-early-termination-policy

Now we are ready to configure a run configuration object, and specify the primary metric 'Accuracy' that's recorded in your training runs. 
If you go back to visit the training script, you will notice that this value is being logged. 
We also want to tell the service that we are looking to maximizing this value. 
We also set the number of samples to 20, and maximal concurrent job to 4.

In [ ]:
# Note that in this case when using HyperDrive, the original ScriptRunConfig's parameters are not used but the HyperDrive parameters...
hyperdrive_config = HyperDriveConfig(run_config=script_runconfig, 
                                     # estimator=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=5,
                                     max_concurrent_runs=4)

Finally, launch the hyperparameter tuning job.

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Check here how to submit the hyperdrive run as a step of an AML Pipeline:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-parameter-tuning-with-hyperdrive.ipynb

### Monitor HyperDrive run

Monitor the progress of the runs with the Jupyter widget.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
# Get details for debugging:
# RunDetails(run).get_widget_data()

#### Get log results upon completion
Model training and monitoring happen in the background. Wait until the model has finished training before you run more code. Use wait_for_completion to show when the model training is finished:

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

### Find and get the best model found by HyperDrive¶ 
When all jobs finish, we can find out the one that has the highest accuracy.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
# print(best_run.get_details())

In [ ]:
# Copy 'best_run' to 'run' to re-use the same code also used without HyperDrive
run = best_run

## Display run metrics results
You now have a model trained on a remote cluster. Retrieve the accuracy of the model:

In [ ]:
print(run.get_metrics())

## See files associated with the run

We can either see the logs for each run in the AzureML studio UI or download them here.

In [ ]:
print(run.get_file_names())

run.download_file('azureml-logs/70_driver_log.txt')

## Register the model
Once you've trained the model, you can save and register it to your workspace. Model registration lets you store and version your models in your workspace to simplify model management and deployment.

Running the following code will register the model to your workspace, and will make it available to reference by name in remote compute contexts or deployment scripts. 

In [ ]:
from azureml.core.model import Model

model_reg = run.register_model(model_name='classif-empl-attrition-aml-comp',    # Name of the registered model in your workspace.
                               description='Binary classification model for employees attrition',
                               model_path='outputs/classif-empl-attrition.pkl', # Local file to upload and register as a model.
                               model_framework=Model.Framework.SCIKITLEARN,     # Framework used to create the model.
                               model_framework_version='0.22.2.post1',                # Version of scikit-learn used to create the model.
                               tags={'ml-task': "binary-classification", 'business-area': "HR"},
                               properties={'joblib-version': "0.14.0", 'pandas-version': "0.24.1"},
                               sample_input_dataset=aml_dataset
                              )
model_reg

### Download Scikit-Learn model pickle file from the run (Option A)

In [ ]:
# Download the model .pkl file to local (Using the 'run' object)
run.download_file('outputs/classif-empl-attrition.pkl')

### Download Scikit-Learn model pickle file from the model registry (Option B)

In [ ]:
print(Model.get_model_path('classif-empl-attrition-aml-comp', _workspace=ws))

model_from_registry = Model(ws,'classif-empl-attrition-aml-comp')
model_from_registry.download(target_dir='.', exist_ok=True)


# Try model predictions in this notebook

### Load model into memory

In [ ]:
# Load the model into memory
model = joblib.load('classif-empl-attrition.pkl')

model

In [ ]:
# Load model and test datasets from .pkl files

# Download the test datasets to local
run.download_file('outputs/x_test.pkl')
run.download_file('outputs/y_test.pkl')

# Load the test datasets into memory
x_test = joblib.load('x_test.pkl')
y_test = joblib.load('y_test.pkl')

# joblib
# https://joblib.readthedocs.io/en/latest/installing.html
# https://joblib.readthedocs.io/en/latest/generated/joblib.load.html

## Make Predictions and calculate Accuracy metric

In [ ]:
from sklearn.metrics import accuracy_score

# Make Multiple Predictions
y_predictions = model.predict(x_test)  # .predict(X[0:1])

y_predictions

print('Accuracy:')
accuracy_score(y_test, y_predictions)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_predictions)

print(cm)

# Show confusion matrix in a separate window
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# One Prediction
instance_num = 6
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = model.predict(x_test)[instance_num]

print("One Prediction: ")
print(prediction_value)

print("20 Predictions: ")
print(y_predictions[:20])

x_test.head(20)

In [ ]:
y_test.head(5)